## Zero-noise extrapolation of QuTiP circuits with the Cirq simulator

In [1]:
import numpy as np
import mitiq
import cirq
from cirq import Circuit, LineQubit, X
import qutip

In [4]:
depth = 8
qbit = LineQubit(0)
circ = Circuit(X(qbit) for _ in range(depth))

### 1. Transforming a quantum circuit from Cirq to QuTiP (Cirq $\rightarrow$ QASM  $\rightarrow$ QuTiP)

Showing it in an example:

In [5]:
from cirq import Circuit, LineQubit, X
from qutip.qip.qasm import *

# create the quantum circuit in Cirq 
qbit = LineQubit(0)
circ = Circuit(X(qbit) for _ in range(8))

# export the quantum circuit to QASM format
qasm_circuit = cirq.qasm(circ)

# import to qutip (`qutip.qip.QubitCircuit()` object)
qutip_circuit = read_qasm(qasm_circuit,strmode=True)

qutip_circuit

Defining a simple conversion function:

In [6]:
from qutip.qip.circuit import QubitCircuit

def cirq_to_qutip(circ: cirq.Circuit) -> QubitCircuit:
    """Transform a cirq circuit into qutip's, using qasm.

    Args:
        circ: The quantum program as a cirq circuit.

    Returns:
        qutip_circ: The quantum program as a qutip circuit, `QubitCircuit`.
    """
    from qutip.qip.qasm import read_qasm
    
    qasm_circuit = cirq.qasm(circ)
    qutip_circuit = read_qasm(qasm_circuit,strmode=True)
    return qutip_circuit

Testing the function:

In [7]:
cirq_to_qutip(circ=circ)

In [8]:
type(cirq_to_qutip(circ=circ))

qutip.qip.circuit.QubitCircuit

### 2. Transforming a quantum circuit from QuTiP to Cirq using Mitiq (QuTiP $\rightarrow$ QASM  $\rightarrow$ Cirq)

In [9]:
from qutip.qip.circuit import QubitCircuit
from mitiq.mitiq_qiskit.conversions import from_qasm

Showing it in an example:

In [10]:
depth = 8
circuit_qutip = QubitCircuit(N=1)
for i in range(0,depth):
    circuit_qutip.add_gate("X", targets=0)
circuit_qutip

In [11]:
qasm_circuit2 = circuit_to_qasm_str(circuit_qutip)
cirq_circuit2 = from_qasm(qasm_circuit2)
cirq_circuit2

q_0: ───X───X───X───X───X───X───X───X───

In [12]:
print_qasm(circuit_qutip)

// QASM 2.0 file generated by QuTiP

OPENQASM 2.0;
include "qelib1.inc";

qreg q[1];

x q[0];
x q[0];
x q[0];
x q[0];
x q[0];
x q[0];
x q[0];
x q[0];


In [13]:
from qutip.qip.circuit import QubitCircuit
from cirq import Circuit

Defining a simple conversion function:

In [14]:
def qutip_to_cirq(circ: QubitCircuit) -> cirq.Circuit:
    """Transform a QuTiP circuit into Cirq's, using Mitiq's QASM converter.

    Args:
        circ: The quantum program as a qutip circuit, `QubitCircuit`.
    Returns:
        cirq_circ: The quantum program as a cirq circuit.
    """
    from qutip.qip.qasm import circuit_to_qasm_str
    from mitiq.mitiq_qiskit.conversions import from_qasm
    qasm_circ = circuit_to_qasm_str(circ)
    cirq_circ = from_qasm(qasm_circ)
    return cirq_circ

In [15]:
qutip_to_cirq(cirq_to_qutip(circ=circ))

q_0: ───X───X───X───X───X───X───X───X───

In [16]:
circ

0: ───X───X───X───X───X───X───X───X───

In [17]:
# Testing the function

### 3. Using Mitiq on a QuTiP circuit
We use Mitiq to mitigate the noisy execution of a QuTiP circuit. We use ``qutip.QubitCircuit`` as input, and then run the simulation on Cirq's simulator. 

In [18]:
from cirq import Circuit, depolarize
from cirq import DensityMatrixSimulator
from mitiq import execute_with_zne

In [19]:
# Observable to measure
z = np.diag([1, -1])
#zz = np.kron(z, z)

# Strength of noise channel
p = 0.05


def executor(circ: Circuit) -> float:
    """
    Simulates the execution of a circuit with depolarizing noise.

    Args:
        circ: The input circuit.

    Returns:
        The expectation value of the ZZ observable.
    """
    # Add depolarizing noise to the circuit
    circuit = circ.with_noise(depolarize(p))

    # Get the final density matrix of the circuit
    rho = SIMULATOR.simulate(circuit).final_density_matrix

    # Evaluate the ZZ expectation value
    expectation = np.real(np.trace(rho @ z))
    return expectation

from cirq import Circuit, CNOT, DensityMatrixSimulator, H, LineQubit, depolarize, rz
from mitiq.zne import mitigate_executor

SIMULATOR = DensityMatrixSimulator()

execute_with_zne(circ,executor)

0.9236830621957784

In [44]:
mitiq.about()
qutip.about()


Mitiq: A Python toolkit for implementing error mitigation on quantum computers
Authored by: Mitiq team, 2020 & later (https://github.com/unitaryfund/mitiq)

Mitiq Version:	0.1.0

Cirq Version:	0.9.0.dev
NumPy Version:	1.18.5
SciPy Version:	1.4.1
PyQuil Version:	2.18.0
Qiskit Version:	0.12.0

Python Version:	3.8.5
Platform Info:	Darwin (x86_64)
Install Path:	/Users/nathanshammah/github/mitiq/mitiq

QuTiP: Quantum Toolbox in Python
Copyright (c) QuTiP team 2011 and later.
Original developers: R. J. Johansson & P. D. Nation.
Previous lead developers: Chris Granade & A. Grimsmo.
Current admin team: Alexander Pitchford, Paul D. Nation, Nathan Shammah, Shahnawaz Ahmed, Neill Lambert, Eric Giguère, and Boxi Li
Project Manager: Franco Nori.
Currently developed through wide collaboration. See https://github.com/qutip for details.

QuTiP Version:      4.6.0.dev0+6941e402
Numpy Version:      1.18.5
Scipy Version:      1.4.1
Cython Version:     0.29.21
Matplotlib Version: 3.2.2
Python Version:   

/miniconda3/envs/mit3i/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
